In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# 基础参数
n_tasks = 100
m_pus = 5
k_chains = 10
tasks_per_chain = 10
K_levels = 5

# 能源等级参数
L_levels = [20, 18, 12, 7, 3]      # 延迟
E_levels = [10, 30, 40, 45, 120]   # 任务能耗
f_v_levels = [20, 30, 40, 60, 80]  # CPU资源

# PU 参数
C_PU = 2000
E_PU_max = 9000
P_idle = [50] * m_pus
P_max = [200] * m_pus
T_total = 400

# 构造任务链
task_chains = []
for k in range(k_chains):
    task_chains.append(list(range(k * tasks_per_chain, (k + 1) * tasks_per_chain)))

# Gurobi 优化函数
def run_gurobi_model(alpha_1=1, alpha_2=1, alpha_3=1):
    model = gp.Model("MAF_Optimization")
    model.Params.LogToConsole = 0

    # 决策变量
    delta = model.addVars(n_tasks, vtype=GRB.BINARY, name="delta")
    w = model.addVars(n_tasks, m_pus, vtype=GRB.BINARY, name="w")
    y = model.addVars(n_tasks, K_levels, vtype=GRB.BINARY, name="y")
    z = model.addVars(k_chains, vtype=GRB.BINARY, name="z")
    E_PU = model.addVars(m_pus, name="E_PU")

    # 1. 每个任务最多一个 PU
    for i in range(n_tasks):
        model.addConstr(gp.quicksum(w[i, j] for j in range(m_pus)) == delta[i])

    # 2. 每个任务只能选择一个能耗等级
    for i in range(n_tasks):
        model.addConstr(gp.quicksum(y[i, k] for k in range(K_levels)) == delta[i])

    # 3. 任务链激活与任务绑定
    for k in range(k_chains):
        for i in task_chains[k]:
            model.addConstr(delta[i] <= z[k])

    # 4. PU 资源限制
    for j in range(m_pus):
        model.addConstr(
            gp.quicksum(
                w[i, j] * gp.quicksum(y[i, k] * f_v_levels[k] for k in range(K_levels))
                for i in range(n_tasks)
            ) <= C_PU
        )

    # 5. PU 能耗模型
    for j in range(m_pus):
        active_time = gp.quicksum(
            w[i, j] * gp.quicksum(y[i, k] * L_levels[k] for k in range(K_levels))
            for i in range(n_tasks)
        )
        U_j = active_time / T_total
        E_PU_expr = (
            P_idle[j] * (T_total - active_time) +
            (P_max[j] - P_idle[j]) * U_j * T_total
        )
        model.addConstr(E_PU[j] == E_PU_expr)
        model.addConstr(E_PU[j] <= E_PU_max)

    # 6. 目标函数
    total_task_energy = gp.quicksum(
        y[i, k] * E_levels[k] for i in range(n_tasks) for k in range(K_levels)
    )
    total_chain_accept = gp.quicksum(z[k] for k in range(k_chains))
    total_pu_energy = gp.quicksum(E_PU[j] for j in range(m_pus))

    model.setObjective(
        alpha_1 * total_chain_accept - alpha_2 * total_task_energy - alpha_3 * total_pu_energy,
        GRB.MAXIMIZE
    )

    # 求解
    model.optimize()

    # 返回结果
    accepted_chains = [k for k in range(k_chains) if z[k].X > 0.5]
    return {
        "objVal": model.ObjVal,
        "accepted_chains": accepted_chains,
        "total_chain_accepted": len(accepted_chains),
        "total_pu_energy": sum(E_PU[j].X for j in range(m_pus)),
        "total_task_energy": sum(y[i, k].X * E_levels[k] for i in range(n_tasks) for k in range(K_levels)),
    }

# 测试运行
results = run_gurobi_model(alpha_1=1, alpha_2=1, alpha_3=1)



Set parameter Username
Set parameter LicenseID to value 2599825
Academic license - for non-commercial use only - expires 2025-12-16
Set parameter LogToConsole to value 0


AttributeError: Unable to retrieve attribute 'X'

In [2]:
import gurobipy as gp
from gurobipy import GRB

def run_gurobi_model(alpha_1=1, alpha_2=1, alpha_3=1):
    n_tasks = 100
    m_pus = 5
    k_chains = 10
    tasks_per_chain = 10
    K_levels = 5

    # 能源等级参数
    L_levels = [20, 18, 12, 7, 3]
    E_levels = [10, 30, 40, 45, 120]
    f_v_levels = [20, 30, 40, 60, 80]

    C_PU = 2000
    E_PU_max = 9000
    P_idle = [50] * m_pus
    P_max = [200] * m_pus
    T_total = 400

    task_chains = [list(range(k * tasks_per_chain, (k + 1) * tasks_per_chain)) for k in range(k_chains)]

    model = gp.Model("MAF_Optimization")
    model.Params.LogToConsole = 1

    delta = model.addVars(n_tasks, vtype=GRB.BINARY, name="delta")
    w = model.addVars(n_tasks, m_pus, vtype=GRB.BINARY, name="w")
    y = model.addVars(n_tasks, K_levels, vtype=GRB.BINARY, name="y")
    z = model.addVars(k_chains, vtype=GRB.BINARY, name="z")
    E_PU = model.addVars(m_pus, name="E_PU")

    # 每个任务最多一个 PU
    for i in range(n_tasks):
        model.addConstr(gp.quicksum(w[i, j] for j in range(m_pus)) == delta[i])

    # 每个任务一个能耗等级
    for i in range(n_tasks):
        model.addConstr(gp.quicksum(y[i, k] for k in range(K_levels)) == delta[i])

    # 任务链激活与任务绑定
    for k in range(k_chains):
        for i in task_chains[k]:
            model.addConstr(delta[i] <= z[k])

    # PU 资源限制
    for j in range(m_pus):
        model.addConstr(
            gp.quicksum(
                w[i, j] * gp.quicksum(y[i, k] * f_v_levels[k] for k in range(K_levels))
                for i in range(n_tasks)
            ) <= C_PU
        )

    # PU 能耗模型
    for j in range(m_pus):
        active_time = gp.quicksum(
            w[i, j] * gp.quicksum(y[i, k] * L_levels[k] for k in range(K_levels))
            for i in range(n_tasks)
        )
        U_j = active_time / T_total
        E_PU_expr = (
            P_idle[j] * (T_total - active_time) +
            (P_max[j] - P_idle[j]) * U_j * T_total
        )
        model.addConstr(E_PU[j] == E_PU_expr)
        model.addConstr(E_PU[j] <= E_PU_max)

    # 目标函数
    total_task_energy = gp.quicksum(
        y[i, k] * E_levels[k] for i in range(n_tasks) for k in range(K_levels)
    )
    total_chain_accept = gp.quicksum(z[k] for k in range(k_chains))
    total_pu_energy = gp.quicksum(E_PU[j] for j in range(m_pus))

    model.setObjective(
        alpha_1 * total_chain_accept - alpha_2 * total_task_energy - alpha_3 * total_pu_energy,
        GRB.MAXIMIZE
    )

    model.optimize()

    accepted_chains = [k for k in range(k_chains) if z[k].X > 0.5]
    print("目标值 =", model.ObjVal)
    print("被接受的任务链数 =", len(accepted_chains))
    print("PU 总能耗 =", sum(E_PU[j].X for j in range(m_pus)))
    print("任务总能耗 =", sum(y[i, k].X * E_levels[k] for i in range(n_tasks) for k in range(K_levels)))

    return model
